# Point-to-point task

In this exercise, we'll solve the point-to-point task for the Talos arm. For this problem, the robot needs to pass through four 3D points, and them to come back to home position (i.e. initial position)

First our main goal is to define a stack of action models inside the PointToPointProblem class. For that we need to define the set of cost functions per each phase. Note that we use a differential model for ABA dynamics along an integrated action model to convert it into discrete-time.

Please see the instruction inside the PointToPointProblem class.

In [ ]:
from crocoddyl import DifferentialActionModel, IntegratedActionModelEuler
import pinocchio
import numpy as np

class PointToPointProblem:
    def __init__(self, robot, frameId):
        self.robot = robot
        self.frameId = frameId
        self.state = StatePinocchio(self.robot.model)
    def createProblem(self, x0, T, dt, p1, p2, p3, p4):
        """ Define 4 points picking points
        """
        # Our robot needs to pass through 4 points and then come back to the default posture
        # First we defined the running action models per each point
        goFirstPointAction = IntegratedActionModelEuler(DifferentialActionModel(self.robot.model))
        goSecondPointAction = IntegratedActionModelEuler(DifferentialActionModel(self.robot.model))
        goThirdPointAction = IntegratedActionModelEuler(DifferentialActionModel(self.robot.model))
        goFourthPointAction = IntegratedActionModelEuler(DifferentialActionModel(self.robot.model))
        goHomeAction = IntegratedActionModelEuler(DifferentialActionModel(self.robot.model))
        # And later the terminal action models per each point
        goFirstPointActionTerm = IntegratedActionModelEuler(DifferentialActionModel(self.robot.model))
        goSecondPointActionTerm = IntegratedActionModelEuler(DifferentialActionModel(self.robot.model))
        goThirdPointActionTerm = IntegratedActionModelEuler(DifferentialActionModel(self.robot.model))
        goFourthPointActionTerm = IntegratedActionModelEuler(DifferentialActionModel(self.robot.model))
        goHomeActionTerm = IntegratedActionModelEuler(DifferentialActionModel(self.robot.model))
        
        # Setting up the time step per each action model
        goFirstPointAction.timeStep = dt
        goSecondPointAction.timeStep = dt
        goThirdPointAction.timeStep = dt
        goFourthPointAction.timeStep = dt
        goHomeAction.timeStep = dt
        

        ######################################################################
        ######### TODO: Add the cost functions per each action model #########
        ######################################################################
        # Hints: Indenpendely of the action, you need to define three cost functions:
        # goToCost, uRegCost and xRegCost. These are the main points:
        #  1. Define the uRegCost and xRegCost.
        #  2. Add these cost function in each action model (e.g. goFirstPointAction)
        #     with weights: 1./1e-3 (terminal/running goToCost), 1e-7 (uRegCost and xRegCost)
        # For adding the cost functions you should do:
        #  myAction.addCost( name="myCostName", weight = myWeight, cost = myCostObject)

        # Define the set of action models (we call it taskModels)
        goFirstPointModels = [ goFirstPointAction ] * T + [ goFirstPointActionTerm ]
        goSecondPointModels = [ goSecondPointAction ] * T + [ goSecondPointActionTerm ]
        goThirdPointModels = [ goThirdPointAction ] * T + [ goThirdPointActionTerm ]
        goFourthPointModels = [ goFourthPointAction ] * T + [ goFourthPointActionTerm ]
        taskModels = goFirstPointModels + goSecondPointModels \
            + goThirdPointModels + goFourthPointModels + goHomeModels

        # Building a shooting problem from a stack of action models
        problem = ShootingProblem(x0, taskModels, goHomeActionTerm)
        return problem
    def goToCost(self, p_ref):
        # Return SE3 tracking cost
        SE3ref = pinocchio.SE3(np.eye(3), p_ref)
        return CostModelPosition6D(self.robot.model,
                                   nu=self.robot.model.nv,
                                   self.frameId,
                                   ref=SE3ref)
    def uRegCost(self):
        # Hints: You can build this control regularization function by using
        # CostModelControl
        return None # control regularization cost
    def xRegCost(self):
        # Hints: You can build this control regularization function by using
        # CostModelState
        return None # state regularization cost

Onces we have defined our point-to-point problem class. Let's solve it with our DDP solver!

In [ ]:
from crocoddyl import SolverDDP
from crocoddyl import CallbackDDPLogger, CallbackDDPVerbose, CallbackSolverDisplay
from crocoddyl import loadTalosArm


# Loading the Talos arm and defining the wripper frame
talos_arm = loadTalosArm()
wripperFrame = talos_arm.model.getFrameId('gripper_left_joint')

# Defining a initial state, Horizon and time step
q0 = [0.173046, 1., -0.52366, 0., 0., 0.1, -0.005]
x0 = np.hstack([q0, np.zeros(talos_arm.model.nv)])
T = 40
dt = 1e-3

# Creating the point-to-point problem
p2p = PointToPointProblem(talos_arm, wripperFrame)
problem = p2p.createProblem(x0, T, dt, p1, p2, p3, p4)

# Creating the DDP solver for this OC problem, defining a logger
ddp = SolverDDP(problem)
cameraTF = [2., 2.68, 0.54, 0.2, 0.62, 0.72, 0.22]
ddp.callback = [CallbackDDPLogger(), CallbackDDPVerbose(1), CallbackSolverDisplay(robot,4,cameraTF)]

# Solving it with the DDP algorithm
ddp.solve()


